In [2]:
import os
import pandas as pd

import torch
import torch.optim as optim
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

import torchvision.models as models # 이미지
from torchvision import transforms # 아마지의 크기, 증강, 정규화를 진행하고 토치텐서로 출력하기 위해 사용
from PIL import Image

from transformers import GPT2Tokenizer, GPT2Model # 텍스트

from tqdm.auto import tqdm
import warnings 
warnings.filterwarnings('ignore')

from Modules.train import train
from Modules.inference import inference
from Modules.VQADataset import VQADataset
from Modules.VQAModel import VQAModel
from Modules.load_data import load_data

# device
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"current device is {device}")

train_df, test_df, sample_submission, train_img_path, test_img_path = load_data()

# dataset & dataloaders
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
# len(tokenizer)는 토크나이저가 인식하는 전체 토큰의 개수를 나타낸다.
# 즉, 토크나이저 어휘집의 크기와 동일합니다. 이 숫자는 모델이 학습된 토크나이저에 따라 달라진다.
# GPT2 토크나이저 어휘집 50258
vocab_size = len(tokenizer)

transform = transforms.Compose([ # 이미지 변환 함수들을 조합하여 하나의 변환 함수로 만들어주는 함수이다.
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])

train_dataset = VQADataset(train_df, tokenizer, transform, train_img_path, is_test=False)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

# Model
model = VQAModel(vocab_size).to(device)

# Criterion and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=5e-5)

# Training loop
for epoch in range(1):
    avg_loss = train(model, train_loader, optimizer, criterion)
    print(f"Epoch: {epoch+1}, Loss: {avg_loss:.4f}") # fprint 처리 잘하기 

current device is cuda
50258
